In [1]:
# import sys
# sys.path.append('./')
# import numpy as np
# import os
# import datetime
# import keras.backend as K
# import tensorflow as tf
# from keras.layers import Conv2D, Dense, DepthwiseConv2D,add
# from keras.optimizers import SGD, Adam
# import numpy as np
# import math
# import keras
# from PIL import Image
# from random import shuffle
# from keras import layers as KL
from Anchors import get_anchors
from Datasets import SSDDatasets
# from learning_rate import WarmUpCosineDecayScheduler
from loss import MultiboxLoss
from Models import SSD300
from utils import get_classes, show_config
# from log_record import record_log, read_log
# from keras.callbacks import (EarlyStopping, LearningRateScheduler,
#                              ModelCheckpoint, TensorBoard)
# from callbacks import (ExponentDecayScheduler, LossHistory,
#                        ParallelModelCheckpoint, EvalCallback)


2024-06-27 20:51:02.254124: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-27 20:51:02.285203: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-27 20:51:02.285655: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-27 20:51:02.988325: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
from keras.regularizers import l2

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping


if __name__ == "__main__":
    
    # 设置训练参数
    Epoch = 500  # 训练100 epochs
    lr = 1e-4  # Adam优化器，所以较小的学习率
    optimizer_type = "Adam"
    momentum = 0.937
    batch_size = 16
    imgcolor = 'grey'  # imgcolor选“rgb” or “grey”, 则处理图像变单通道或者三通道
    tmp_dir = str(datetime.datetime.strftime(datetime.datetime.now(), '%Y%m%d'))
    save_dir = "/home/zhangyouan/桌面/zya/NN_net/network/SSD/IMX_681_ssd_mobilenet_git/keras/detection/SSD_ipynb_transfer_callback_1227hand/output/20240228"
        
    # 设置SSD参数
    cls_name_path = "./model_data/voc_classes.txt"  # 导入目标检测类别；
    input_shape = [120, 160]  # 输入的尺寸大小
    anchor_size = [32, 59, 86, 113, 141, 168]  # 用于设定先验框的大小，根据公式计算而来；如果要检测小物体，修改浅层先验框的大小，越小的话，识别的物体越小；    
    train_annotation_path = r'/home/zhangyouan/桌面/zya/dataset/681/hand/2007_train.txt'  # 训练图片路径和标签
    val_annotation_path = r'/home/zhangyouan/桌面/zya/dataset/681/hand/2007_test.txt'  # 验证图片路径和标签
        
    # 1. 获取classes和anchor
    class_names, num_cls = get_classes(cls_name_path)
    num_cls += 1  # 增加一个背景类别
    print("class_names:", class_names, "num_classes:", num_cls)
    
    # 2. 获取anchors, 输出的是归一化之后的anchors
    anchor = get_anchors(input_shape, anchor_size)
    print("type:",type(anchor), "shape:", np.shape(anchor))

    # 3. 模型编译
    K.clear_session()
    model_path = ""
    # model_path = "./output/20230804_3/good_detection_test_callback.h5"
    model = SSD300((input_shape[0], input_shape[1], 1), num_cls)
    # model.save("template.h5")
    # model.summary()
    if model_path != "":
        model.load_weights(model_path, by_name = True, skip_mismatch=True)
       
    # 4. 优化器
    # optimizer = Adam(lr = lr, beta_1=momentum)
    # optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    from tensorflow.keras.optimizers import legacy
    optimizer = legacy.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.0)
    
    # 5. 导入数据集
    with open(train_annotation_path, encoding='utf-8') as f:
        train_lines = f.readlines()
    with open(val_annotation_path, encoding='utf-8') as f:
        val_lines = f.readlines()
    num_train = len(train_lines)
    num_val = len(val_lines)
    epoch_step = num_train // batch_size
    epoch_step_val = num_val // batch_size
    
    # 数据增强设置
    train_dataloader = SSDDatasets(train_lines, input_shape, anchor, batch_size, num_cls, train=False, imgcolor=imgcolor)
    val_dataloader = SSDDatasets(val_lines, input_shape, anchor, batch_size, num_cls, train=False, imgcolor=imgcolor)
    print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
    
    # 6. 编译模型
    # losses = {'mbox_loc_final':MultiboxLoss(num_cls, neg_pos_ratio=3.0).compute_loc_loss,
            #   'cls_conf_final':MultiboxLoss(num_cls, neg_pos_ratio=3.0).compute_conf_loss}
    # losses2 = [MultiboxLoss(num_cls, neg_pos_ratio=3.0).compute_loc_loss,MultiboxLoss(num_cls, neg_pos_ratio=3.0).compute_conf_loss]
    model.compile(optimizer=optimizer, loss = MultiboxLoss(num_cls, neg_pos_ratio=3.0).compute_loss)
    
    # # 8. 精度评价: pending --> 还没构建；
    # eval_flag = True
    # eval_period = 10
    # eval_callback = EvalCallback(model, input_shape, anchor, class_names, num_cls, val_lines, log_dir, eval_flag=eval_flag, period = eval_period)
    show_config(
        classes_path=cls_name_path, model_path=model_path, input_shape=input_shape, \
        Epoch=Epoch, batch_size=batch_size, \
        lr=lr, optimizer_type=optimizer_type, momentum=momentum, \
        num_train=num_train, num_val=num_val
    )
    checkpoint = ModelCheckpoint('ssd_weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=20, verbose=1)
    csv_logger = CSVLogger('training.log')
    early_stopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1)

    # 8. 开始训练；
    history = model.fit(
            train_dataloader,
            validation_data=val_dataloader,
            steps_per_epoch=epoch_step,
            validation_steps=epoch_step_val,
            epochs=Epoch,
            callbacks=[checkpoint,reduce_lr, csv_logger, early_stopping, 
                       keras.callbacks.TensorBoard(log_dir=os.path.join(save_dir, 'unetlogs'), update_freq=1000)],
        )
    
    record_log(history, filename = os.path.join(save_dir, "unetlogs/log.txt"))
    model.save(os.path.join(save_dir, "hand_detection_20240626_gpt.h5"))
    model.save(os.path.join(save_dir, "hand_detection_20240626_gpt.pb"))

class_names: ['hand'] num_classes: 2
type: <class 'numpy.ndarray'> shape: (1242, 4)
Train on 1440 samples, val on 178 samples, with batch size 16.
Configurations:
----------------------------------------------------------------------
|                     keys |                                   values|
----------------------------------------------------------------------
|             classes_path |             ./model_data/voc_classes.txt|
|               model_path |                                         |
|              input_shape |                               [120, 160]|
|                    Epoch |                                      500|
|               batch_size |                                       16|
|                       lr |                                   0.0001|
|           optimizer_type |                                     Adam|
|                 momentum |                                    0.937|
|                num_train |                            

2024-06-27 20:10:45.427066: W tensorflow/c/c_api.cc:304] Operation '{name:'DepthwiseConv2D_conf_DD5_1/bias/Assign' id:1484 op device:{requested: '', assigned: ''} def:{{{node DepthwiseConv2D_conf_DD5_1/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](DepthwiseConv2D_conf_DD5_1/bias, DepthwiseConv2D_conf_DD5_1/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


 4/90 [>.............................] - ETA: 5s - batch: 1.5000 - size: 16.0000 - loss: 7.1671     

2024-06-27 20:10:46.695305: I tensorflow/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-06-27 20:10:46.695364: I tensorflow/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-06-27 20:10:46.756634: I tensorflow/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2024-06-27 20:10:46.761818: I tensorflow/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2024-06-27 20:10:46.764167: I tensorflow/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: /home/zhangyouan/桌面/zya/NN_net/network/SSD/IMX_681_ssd_mobilenet_git/keras/detection/SSD_ipynb_transfer_callback_1227hand/output/20240228/unetlogs/plugins/profile/2024_06_27_20_10_46/SSSLXSRVLS001.xplane.pb


90/90 [==============================] - ETA: 0s - batch: 44.5000 - size: 16.0000 - loss: 6.7485

/home/zhangyouan/anaconda3/envs/stc/lib/python3.10/site-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
2024-06-27 20:10:51.917921: W tensorflow/c/c_api.cc:304] Operation '{name:'loss/AddN' id:1775 op device:{requested: '', assigned: ''} def:{{{node loss/AddN}} = AddN[N=2, T=DT_FLOAT, _has_manual_control_dependencies=true](loss/mul, loss/Conv2D_layer13/kernel/Regularizer/mul)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


90/90 [==============================] - 12s 96ms/step - batch: 44.5000 - size: 16.0000 - loss: 6.7485 - val_loss: 6.1902 - lr: 0.0010
Epoch 2/500
90/90 [==============================] - 6s 67ms/step - batch: 44.5000 - size: 16.0000 - loss: 5.8294 - val_loss: 5.6375 - lr: 0.0010
Epoch 3/500
90/90 [==============================] - 6s 65ms/step - batch: 44.5000 - size: 16.0000 - loss: 5.5655 - val_loss: 5.4875 - lr: 0.0010
Epoch 4/500
90/90 [==============================] - 6s 65ms/step - batch: 44.5000 - size: 16.0000 - loss: 5.4713 - val_loss: 5.4503 - lr: 0.0010
Epoch 5/500
90/90 [==============================] - 6s 65ms/step - batch: 44.5000 - size: 16.0000 - loss: 5.4450 - val_loss: 5.4170 - lr: 0.0010
Epoch 6/500
90/90 [==============================] - 6s 64ms/step - batch: 44.5000 - size: 16.0000 - loss: 5.4313 - val_loss: 5.3899 - lr: 0.0010
Epoch 7/500
90/90 [==============================] - 6s 66ms/step - batch: 44.5000 - size: 16.0000 - loss: 5.4176 - val_loss: 5.3685 - 

/home/zhangyouan/anaconda3/envs/stc/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


INFO:tensorflow:Assets written to: /home/zhangyouan/桌面/zya/NN_net/network/SSD/IMX_681_ssd_mobilenet_git/keras/detection/SSD_ipynb_transfer_callback_1227hand/output/20240228/hand_detection_20240626_gpt.pb/assets


INFO:tensorflow:Assets written to: /home/zhangyouan/桌面/zya/NN_net/network/SSD/IMX_681_ssd_mobilenet_git/keras/detection/SSD_ipynb_transfer_callback_1227hand/output/20240228/hand_detection_20240626_gpt.pb/assets


In [ ]:
from log_record import record_log, read_log
from utils import visual_train

history = read_log("/home/zhangyouan/桌面/zya/NN_net/network/SSD/IMX_681_ssd_mobilenet_git/keras/detection/SSD_ipynb_transfer_callback/output/20230819/unetlogs/log.txt")
visual_train(history)

no accuracy, only loss.


In [6]:
from log_record import record_log, read_log
from utils import visual_train

history = read_log("/home/zhangyouan/桌面/zya/NN_net/network/SSD/IMX_681_ssd_mobilenet_git/keras/detection/SSD_ipynb_transfer_callback_0919_cola/output/20231205/unetlogs/log.txt")
visual_train(history)

FileNotFoundError: [Errno 2] No such file or directory: '/home/zhangyouan/桌面/zya/NN_net/network/SSD/IMX_681_ssd_mobilenet_git/keras/detection/SSD_ipynb_transfer_callback_0919_cola/output/20231205/unetlogs/log.txt'

In [12]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
from keras.regularizers import l2


def main():
    try:
        # 设置训练参数
        Epoch = 1000  # 训练1000 epochs
        lr = 1e-4  # Adam优化器的学习率
        momentum = 0.937
        batch_size = 16
        imgcolor = 'grey'  # 图像处理的颜色空间

        # 设置SSD参数
        # cls_name_path = '/home/zhangyouan/桌面/zya/NN_net/network/SSD/IMX_681_ssd_mobilenet_git/keras/detection/VOC_dataset/voc_classes.txt'  # 类别文件路径
        cls_name_path = r"/home/zhangyouan/桌面/zya/NN_net/network/SSD/IMX_681_ssd_mobilenet_git/keras/detection/VOC_dataset/voc_classes.txt"
        input_shape = [120, 160]  # 输入尺寸
        anchor_size = [32, 59, 86, 113, 141, 168]  # 先验框大小    
        # train_annotation_path = '/home/zhangyouan/桌面/zya/NN_net/network/SSD/IMX_681_ssd_mobilenet_git/keras/detection/VOC_dataset/2007_train.txt'
        # val_annotation_path = '/home/zhangyouan/桌面/zya/NN_net/network/SSD/IMX_681_ssd_mobilenet_git/keras/detection/VOC_dataset/2007_val.txt'
        # train_annotation_path = r'/home/zhangyouan/桌面/zya/dataset/681/hand/2007_train.txt'
        # val_annotation_path = r'/home/zhangyouan/桌面/zya/dataset/681/hand/2007_test.txt'
        train_annotation_path = r"/home/zhangyouan/桌面/zya/NN_net/network/SSD/IMX_681_ssd_mobilenet_git/keras/detection/VOC_dataset/2007_train.txt"
        val_annotation_path = r"/home/zhangyouan/桌面/zya/NN_net/network/SSD/IMX_681_ssd_mobilenet_git/keras/detection/VOC_dataset/2007_val.txt"
        # 1. 获取classes和anchors
        class_names, num_cls = get_classes(cls_name_path)
        num_cls += 1  # 增加背景类别
        print("class_names:", class_names, "num_classes:", num_cls)
        
        # 2. 获取anchors
        anchor = get_anchors(input_shape, anchor_size)
        print("type:", type(anchor), "shape:", np.shape(anchor))

        # 3. 清理session并创建模型
        K.clear_session()
        model = SSD300((input_shape[0], input_shape[1], 1), num_cls)
        
        # 4. 设置优化器
        optimizer = Adam(lr=lr, beta_1=momentum)
        
        # 5. 导入数据集
        with open(train_annotation_path, encoding='utf-8') as f:
            train_lines = f.readlines()
        with open(val_annotation_path, encoding='utf-8') as f:
            val_lines = f.readlines()
        num_train = len(train_lines)
        num_val = len(val_lines)
        epoch_step = num_train // batch_size
        epoch_step_val = num_val // batch_size

        # 数据增强设置
        train_dataloader = SSDDatasets(train_lines, input_shape, anchor, batch_size, num_cls, train=True, imgcolor=imgcolor)
        val_dataloader = SSDDatasets(val_lines, input_shape, anchor, batch_size, num_cls, train=False, imgcolor=imgcolor)
        print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
        
        # 6. 编译模型
        model.compile(optimizer=optimizer, loss=MultiboxLoss(num_cls, neg_pos_ratio=3.0).compute_loss)
        # 7. 预训练权重
        model.load_weights(r"/home/zhangyouan/桌面/zya/NN_net/network/SSD/IMX_681_ssd_mobilenet_git/keras/detection/SSD_ipynb_transfer_callback_1227hand/final_det_good_train_1_0860_val_1_0166.h5")
        # 7. 设置回调函数
        checkpoint = ModelCheckpoint('ssd_weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, verbose=1)
        csv_logger = CSVLogger('training.log')
        early_stopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1)

        # 8. 训练模型
        model.fit(
            train_dataloader,
            validation_data=val_dataloader,
            steps_per_epoch=epoch_step,
            validation_steps=epoch_step_val,
            epochs=Epoch,
            callbacks=[checkpoint, reduce_lr, csv_logger, early_stopping]
        )

        # 9. 保存最终模型
        model.save("final_detection_model_det_good_tmp.h5")
        print("Model training completed and saved successfully.")

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


class_names: ['good'] num_classes: 2
type: <class 'numpy.ndarray'> shape: (1242, 4)


Train on 346 samples, val on 346 samples, with batch size 16.
Epoch 1/1000
21/21 [==============================] - 6s 79ms/step - loss: 1.3499 - val_loss: 1.2268 - lr: 0.0010
Epoch 2/1000
21/21 [==============================] - 1s 57ms/step - loss: 1.3878 - val_loss: 1.1425 - lr: 0.0010
Epoch 3/1000
21/21 [==============================] - 1s 55ms/step - loss: 1.3449 - val_loss: 1.1460 - lr: 0.0010
Epoch 4/1000
21/21 [==============================] - 1s 55ms/step - loss: 1.1994 - val_loss: 1.1620 - lr: 0.0010
Epoch 5/1000
21/21 [==============================] - 1s 57ms/step - loss: 1.2621 - val_loss: 1.1304 - lr: 0.0010
Epoch 6/1000
21/21 [==============================] - 1s 54ms/step - loss: 1.2656 - val_loss: 1.1756 - lr: 0.0010
Epoch 7/1000
21/21 [==============================] - 1s 55ms/step - loss: 1.3280 - val_loss: 1.1635 - lr: 0.0010
Epoch 8/1000
21/21 [==============================] - 1s 55ms/step - loss: 1.2918 - val_loss: 1.1914 - lr: 0.0010
Epoch 9/1000
21/21 [======